# Visualización datos de actividad en cursos de Moodle

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os
import sys

current_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
import pandas as pd
import plotly.graph_objects as go

# --- Cargar datos ---
df = pd.read_csv("../../data/interim/moodle/moodle_course_activity_summary.csv")
df_enrollments = pd.read_csv("../../data/interim/estudiantes/enrollments.csv")

# --- Merge ---
df = df.merge(
    df_enrollments[['moodle_user_id', 'id_grado', 'year', 'sede']],
    left_on=['userid', 'year'],
    right_on=['moodle_user_id', 'year'],
    how="left"
)

# --- Variables de eventos ---
event_columns = df.columns.difference(['userid', 'courseid', 'period', 'id_asignatura', 'year', 'course_name', 'sede', 'id_grado', 'moodle_user_id'])

# --- Calcular indicadores generales ---
df['total_eventos'] = df[event_columns].sum(axis=1)

# Total de eventos por sede
total_eventos = df.groupby('sede')['total_eventos'].sum()

# Número de estudiantes únicos por sede
n_estudiantes = df.groupby('sede')['userid'].nunique()

# Promedio de eventos por estudiante
promedio_eventos_estudiante = total_eventos / n_estudiantes

# Total de eventos por periodo y sede
eventos_por_periodo = df.groupby(['period', 'sede'])['total_eventos'].sum().reset_index()

# --- Crear Cards ---
fig = go.Figure()

# Card 1: Total eventos Fusagasugá
fig.add_trace(go.Indicator(
    mode="number",
    value=total_eventos['Fusagasugá'],
    title={"text": "🔵 Total Eventos Fusagasugá"},
    domain={'row': 0, 'column': 0}
))

# Card 2: Total eventos Girardot
fig.add_trace(go.Indicator(
    mode="number",
    value=total_eventos['Girardot'],
    title={"text": "🌸 Total Eventos Girardot"},
    domain={'row': 0, 'column': 1}
))

# Card 3: Promedio eventos Fusagasugá
fig.add_trace(go.Indicator(
    mode="number",
    value=promedio_eventos_estudiante['Fusagasugá'],
    title={"text": "🔵 Promedio Eventos por Estudiante Fusagasugá"},
    domain={'row': 1, 'column': 0}
))

# Card 4: Promedio eventos Girardot
fig.add_trace(go.Indicator(
    mode="number",
    value=promedio_eventos_estudiante['Girardot'],
    title={"text": "🌸 Promedio Eventos por Estudiante Girardot"},
    domain={'row': 1, 'column': 1}
))

fig.update_layout(
    grid={'rows': 2, 'columns': 2, 'pattern': "independent"},
    template="plotly_white",
    title="Resumen de Actividad Moodle",
    title_font_size=24
)

fig.show()

# --- Gráfico adicional de eventos por periodo ---
fig2 = px.bar(
    eventos_por_periodo,
    x='period',
    y='total_eventos',
    color='sede',
    barmode='group',
    title='Total de Eventos por Periodo y Sede',
    labels={'period': 'Periodo', 'total_eventos': 'Total de Eventos'},
    color_discrete_map={'Fusagasugá': 'blue', 'Girardot': 'pink'}
)

fig2.update_layout(
    template="plotly_white",
    title_font_size=24,
    xaxis_title_font_size=18,
    yaxis_title_font_size=18,
    legend_title="Sede",
    legend_font_size=14
)

fig2.show()


In [1]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# --- Cargar datos ---
df = pd.read_csv("../../data/interim/moodle/moodle_course_activity_summary.csv")
df_enrollments = pd.read_csv("../../data/interim/estudiantes/enrollments.csv")
df_asignaturas = pd.read_csv("../../data/raw/tablas_maestras/asignaturas.csv")

# --- Merge de datos ---
df = df.merge(
    df_enrollments[['moodle_user_id', 'id_grado', 'year', 'sede']],
    left_on=['userid', 'year'],
    right_on=['moodle_user_id', 'year'],
    how="left"
)

df = df.merge(
    df_asignaturas[['id_asignatura', 'nombre_2024']],
    on="id_asignatura",
    how="left"
)

# --- Asignar área agrupada ---
def asignar_area(id_asignatura):
    if id_asignatura in [1, 20]:
        return "Ciencias Naturales"
    elif id_asignatura in [2, 19]:
        return "Ciencias Sociales"
    elif id_asignatura in [3, 18, 21]:
        return "Matemáticas"
    else:
        return "Otra"

df['area_final'] = df['id_asignatura'].apply(asignar_area)

# --- Grupos de eventos ---
grupos_eventos = {
    'Interacción Curso': [
        'course_viewed', 'page_module_viewed', 'resource_module_viewed', 'url_module_viewed', 'lti_module_viewed',
        'hvp_module_viewed', 'chat_module_viewed', 'feedback_module_viewed', 'choice_module_viewed', 'hvp_attempt_submitted', 'choice_answer_created', 'feedback_response_submitted'
    ],
    'Evaluaciones': [
        'quiz_module_viewed', 'quiz_attempt_started', 'quiz_attempt_submitted', 'quiz_attempt_summary_viewed',
        'quiz_attempt_viewed', 'quiz_attempt_reviewed', 'assign_submission_form_viewed', 'assign_assessable_submitted',
        'assign_feedback_viewed', 'assign_module_viewed'
    ],
    'Tareas': [
        'file_submission_created', 'file_assessable_uploaded', 'onlinetext_submission_created',
        'onlinetext_assessable_uploaded', 'submission_comment_created', 'submission_status_viewed',
        'assign_remove_submission_form_viewed'
    ],
    'Foros y Comunicación': [
        'forum_module_viewed', 'forum_discussion_viewed', 'forum_post_created', 'forum_post_updated',
        'forum_post_deleted', 'forum_discussion_created', 'forum_discussion_deleted',
        'forum_discussion_subscription_created', 'forum_discussion_subscription_deleted',
        'forum_course_searched', 'forum_subscription_created'
    ],
}

# --- Sumar actividad por grupo ---
for grupo, eventos in grupos_eventos.items():
    df[grupo] = df[eventos].sum(axis=1)

# --- Dataset reducido ---
df_summary = df[['userid', 'id_grado', 'sede', 'area_final', 'year', 'period'] + list(grupos_eventos.keys())]

def graficar_eventos_por_grupo(area_seleccionada=None, grado_seleccionado=None):
    if area_seleccionada is None:
        area_seleccionada = "Todos"
    if grado_seleccionado is None:
        grado_seleccionado = "Todos"
    df_filtrado = df_summary.copy()

    if area_seleccionada != "Todos":
        df_filtrado = df_filtrado[df_filtrado['area_final'] == area_seleccionada]
    if grado_seleccionado != "Todos":
        df_filtrado = df_filtrado[df_filtrado['id_grado'] == grado_seleccionado]

    if df_filtrado.empty:
        print("⚠️ No hay datos para esta combinación.")
        return

    # Agrupar
    total_eventos_grupo = df_filtrado.groupby('sede')[list(grupos_eventos.keys())].sum().reset_index()

    # Convertir a formato largo
    total_eventos_grupo_melted = total_eventos_grupo.melt(id_vars='sede', var_name='Grupo de Eventos', value_name='Total de Eventos')

    fig = px.bar(
        total_eventos_grupo_melted,
        x='Grupo de Eventos',
        y='Total de Eventos',
        color='sede',
        barmode='group',
        title=f'Eventos Totales por Grupo {"- " + area_seleccionada if area_seleccionada != "Todos" else ""} {"- Grado " + str(grado_seleccionado) if grado_seleccionado != "Todos" else ""}',
        labels={'Grupo de Eventos': 'Grupo de Eventos', 'Total de Eventos': 'Cantidad de Eventos'},
        color_discrete_map={'Fusagasugá': 'blue', 'Girardot': 'pink'}
    )

    fig.update_layout(
        template="plotly_white",
        title_font_size=24,
        xaxis_title_font_size=18,
        yaxis_title_font_size=18,
        legend_title="Sede",
        legend_font_size=14,
        xaxis_tickangle=-45
    )

    fig.show()

# --- Crear widgets (Dropdowns) ---
areas = ["Todos"] + sorted(df['area_final'].unique())
grados = ["Todos"] + sorted(df['id_grado'].dropna().unique())

area_dropdown = widgets.Dropdown(
    options=areas,
    value="Todos",
    description='Área:',
    style={'description_width': 'initial'}
)

grado_dropdown = widgets.Dropdown(
    options=grados,
    value="Todos",
    description='Grado:',
    style={'description_width': 'initial'}
)

# --- Función para actualizar gráfico ---
def actualizar_grafico(change=None):
    graficar_eventos_por_grupo(area_dropdown.value, grado_dropdown.value)

area_dropdown.observe(actualizar_grafico, names='value')
grado_dropdown.observe(actualizar_grafico, names='value')

# --- Mostrar widgets y gráfica inicial ---
display(area_dropdown, grado_dropdown)
graficar_eventos_por_grupo()


Dropdown(description='Área:', options=('Todos', 'Ciencias Naturales', 'Ciencias Sociales', 'Matemáticas', 'Otr…

Dropdown(description='Grado:', options=('Todos', np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64…

In [14]:
# Cargar datos
df = pd.read_csv("../../data/interim/moodle/moodle_course_activity_summary.csv")
df_enrollments = pd.read_csv("../../data/interim/estudiantes/enrollments.csv")
df_asignaturas = pd.read_csv("../../data/raw/tablas_maestras/asignaturas.csv")

# --- Merge ---
df = df.merge(
    df_enrollments[['moodle_user_id', 'id_grado', 'year', 'sede']],
    left_on=['userid', 'year'],
    right_on=['moodle_user_id', 'year'],
    how="left"
)

df = df.merge(
    df_asignaturas[['id_asignatura', 'nombre_2024']],
    on="id_asignatura",
    how="left"
)


# --- Crear dataset de eventos por estudiante ---
event_columns = df.columns.difference(['userid', 'courseid', 'period', 'id_asignatura', 'year', 'course_name', 'sede', 'id_grado', 'moodle_user_id', 'nombre_2024'])

# Sumamos eventos por estudiante (usuario) en su grado y sede
actividad_usuario = df.groupby(['userid', 'id_grado', 'sede'])[event_columns].sum(numeric_only=True).reset_index()

# Sumar todos los eventos de cada usuario
actividad_usuario['total_eventos'] = actividad_usuario[event_columns].sum(axis=1)

# Ahora calcular el promedio de eventos por grado y sede
actividad_grado_sede = actividad_usuario.groupby(['id_grado', 'sede'])['total_eventos'].mean().reset_index()

# --- Graficar ---
fig = px.bar(
    actividad_grado_sede,
    x='id_grado',
    y='total_eventos',
    color='sede',
    barmode='group',  # barras lado a lado
    title='Promedio de Actividad por Estudiante - Comparación de Sedes',
    labels={'id_grado': 'Grado', 'total_eventos': 'Promedio de Eventos'},
    color_discrete_map={
        'Fusagasugá': 'blue',
        'Girardot': 'pink'
    }
)

fig.update_layout(
    template="plotly_white",
    title_font_size=24,
    xaxis_title_font_size=18,
    yaxis_title_font_size=18,
    legend_title="Sede",
    legend_font_size=14,
    xaxis=dict(
        type='linear',
        tickmode='linear',
        dtick=1,
    )
)

fig.show()


In [6]:
import pandas as pd
import plotly.express as px

# --- Cargar datos ---
df = pd.read_csv("../../data/interim/moodle/moodle_course_activity_summary.csv")
df_enrollments = pd.read_csv("../../data/interim/estudiantes/enrollments.csv")
df_asignaturas = pd.read_csv("../../data/raw/tablas_maestras/asignaturas.csv")

# --- Merge ---
df = df.merge(
    df_enrollments[['moodle_user_id', 'id_grado', 'year', 'sede']],
    left_on=['userid', 'year'],
    right_on=['moodle_user_id', 'year'],
    how="left"
)

df = df.merge(
    df_asignaturas[['id_asignatura', 'nombre_2024']],
    on="id_asignatura",
    how="left"
)

# --- Variables ---
event_columns = df.columns.difference(['userid', 'courseid', 'period', 'id_asignatura', 'year', 'course_name', 'sede', 'id_grado', 'moodle_user_id', 'nombre_2024'])

# --- Crear nuevas columnas agrupadas por categoría ---
df['navegacion_moodle'] = df[
    ['course_viewed', 'page_module_viewed', 'url_module_viewed', 'resource_module_viewed', 'lti_module_viewed', 'forum_course_searched']
].sum(axis=1)

df['actividades_evaluativas'] = df[
    ['quiz_attempt_started', 'quiz_attempt_submitted', 'quiz_attempt_summary_viewed', 'quiz_attempt_reviewed',
     'quiz_attempt_viewed', 'quiz_module_viewed', 'hvp_attempt_submitted', 'hvp_module_viewed']
].sum(axis=1)

df['tareas_y_entregas'] = df[
    ['assign_module_viewed', 'assign_submission_form_viewed', 'assign_remove_submission_form_viewed',
     'assign_assessable_submitted', 'assign_feedback_viewed',
     'file_assessable_uploaded', 'file_submission_created',
     'onlinetext_assessable_uploaded', 'onlinetext_submission_created',
     'submission_comment_created', 'submission_status_viewed']
].sum(axis=1)

df['participacion_social'] = df[
    ['forum_module_viewed', 'forum_post_created', 'forum_post_updated', 'forum_post_deleted',
     'forum_discussion_created', 'forum_discussion_deleted', 'forum_discussion_viewed',
     'forum_discussion_subscription_created', 'forum_discussion_subscription_deleted',
     'forum_assessable_uploaded', 'forum_subscription_created', 'chat_module_viewed']
].sum(axis=1)

df['feedback_y_elecciones'] = df[
    ['feedback_module_viewed', 'feedback_response_submitted', 'choice_module_viewed', 'choice_answer_created']
].sum(axis=1)

# --- Hacer el heatmap por sede ---
for sede_actual in ['Fusagasugá', 'Girardot']:
    df_sede = df[df['sede'] == sede_actual]

    actividad_categoria_sede = df_sede.groupby('id_grado')[
        ['navegacion_moodle', 'actividades_evaluativas', 'tareas_y_entregas', 'participacion_social', 'feedback_y_elecciones']
    ].sum()

    fig = px.imshow(
        actividad_categoria_sede.T,  # Transponer para grados en X
        labels=dict(x="Grado", y="Categoría de Actividad", color="Número de Eventos"),
        x=actividad_categoria_sede.index,
        y=actividad_categoria_sede.columns,
        title=f"Mapa de Calor de Actividad por Categorías - {sede_actual}",
        color_continuous_scale="Blues"
    )

    fig.update_layout(
        template="plotly_white",
        xaxis_title="Grado",
        yaxis_title="Categoría de Actividad",
        title_font_size=24,
        xaxis=dict(
            type='linear',
            tickmode='linear',
            dtick=1,
        )
    )

    fig.show()


In [8]:
import pandas as pd
import plotly.express as px

# --- Cargar datos ---
df = pd.read_csv("../../data/interim/moodle/moodle_course_activity_summary.csv")
df_enrollments = pd.read_csv("../../data/interim/estudiantes/enrollments.csv")
df_asignaturas = pd.read_csv("../../data/raw/tablas_maestras/asignaturas.csv")

# --- Merge ---
df = df.merge(
    df_enrollments[['moodle_user_id', 'id_grado', 'year', 'sede']],
    left_on=['userid', 'year'],
    right_on=['moodle_user_id', 'year'],
    how="left"
)

df = df.merge(
    df_asignaturas[['id_asignatura', 'nombre_2024']],
    on="id_asignatura",
    how="left"
)

# --- Variables de eventos ---
event_columns = df.columns.difference([
    'userid', 'courseid', 'period', 'id_asignatura', 'year',
    'course_name', 'sede', 'id_grado', 'moodle_user_id', 'nombre_2024'
])

# --- Crear nuevas columnas de categorías ---
df['navegacion_moodle'] = df[
    ['course_viewed', 'page_module_viewed', 'url_module_viewed', 'resource_module_viewed', 'lti_module_viewed', 'forum_course_searched']
].sum(axis=1)

df['actividades_evaluativas'] = df[
    ['quiz_attempt_started', 'quiz_attempt_submitted', 'quiz_attempt_summary_viewed', 'quiz_attempt_reviewed',
     'quiz_attempt_viewed', 'quiz_module_viewed', 'hvp_attempt_submitted', 'hvp_module_viewed']
].sum(axis=1)

df['tareas_y_entregas'] = df[
    ['assign_module_viewed', 'assign_submission_form_viewed', 'assign_remove_submission_form_viewed',
     'assign_assessable_submitted', 'assign_feedback_viewed',
     'file_assessable_uploaded', 'file_submission_created',
     'onlinetext_assessable_uploaded', 'onlinetext_submission_created',
     'submission_comment_created', 'submission_status_viewed']
].sum(axis=1)

df['participacion_social'] = df[
    ['forum_module_viewed', 'forum_post_created', 'forum_post_updated', 'forum_post_deleted',
     'forum_discussion_created', 'forum_discussion_deleted', 'forum_discussion_viewed',
     'forum_discussion_subscription_created', 'forum_discussion_subscription_deleted',
     'forum_assessable_uploaded', 'forum_subscription_created', 'chat_module_viewed']
].sum(axis=1)

df['feedback_y_elecciones'] = df[
    ['feedback_module_viewed', 'feedback_response_submitted', 'choice_module_viewed', 'choice_answer_created']
].sum(axis=1)

# --- Lista de las nuevas categorías ---
categorias = ['navegacion_moodle', 'actividades_evaluativas', 'tareas_y_entregas', 'participacion_social', 'feedback_y_elecciones']

# --- Graficar boxplots ---
for categoria in categorias:
    fig = px.box(
        df,
        x='id_grado',
        y=categoria,
        color='sede',
        title=f'Distribución de {categoria.replace("_", " ").title()} por Grado y Sede',
        labels={'id_grado': 'Grado', categoria: 'Número de Eventos'},
        color_discrete_map={
            'Fusagasugá': 'blue',
            'Girardot': 'pink'
        },
        points="all"  # Para mostrar también los puntos individuales
    )

    fig.update_layout(
        template="plotly_white",
        boxmode="group",  # Para comparar lado a lado
        title_font_size=24,
        xaxis_title_font_size=18,
        yaxis_title_font_size=18,
        legend_title="Sede",
        legend_font_size=14,
        xaxis=dict(
            type='linear',
            tickmode='linear',
            dtick=1
        )
    )

    fig.show()
